# Data Scientist Challenge - LATAM Airlines

## Instrucciones

En Advanced Analytics valoramos muchísimo el trabajo en equipo y la constante interacción entre los distintos roles que trabajan en un producto basado en datos, como el Data Scientist, Machine Learning Engineering, Data Engineer, entre otros. Es por este motivo que una habilidad esencial que buscamos a la hora de buscar nuevos integrantes es el manejo adecuado de git. Este desafío deberá ser entregado en la plataforma de git que más te acomode y que sea pública para que la podamos revisar. Lo que buscamos con esto es poder entender de mejor manera el desarrollo que generaste con tu código, cómo lo fuiste mejorando en el tiempo y si tienes proyectos propios en este repositorio nos servirán para conocer mejor tu experiencia en base a tu propio
portafolio.

Instrucciones Git:
1) Crear un repositorio en la plataforma de git que más te acomode y que sea pública
2) Haber trabajado con una rama principal y otra de desarrollo. Opcional, ocupar alguna práctica de desarrollo de GitFlow.
Instrucciones del desafío:
1) Debes enviar el link al repositorio al mail del que fuiste contactado con asunto Challenge Data Scientist - [Nombre][Apellido], ejemplo Challenge Data Scientist - Pedro Pica Piedra.
2) Se aceptará los cambios en el repositorio hasta la fecha y hora que se indique en el mail.
3) En la siguiente carpeta de Google Drive encontrarás las instrucciones del desafío y el archivo `dataset_SCL.csv` que utilizarás para desarrollarlo.
4) El repositorio debe tener un jupyter notebook llamado solution.ipynb utilizando python 3. No serán revisados otros
lenguajes como R o similar.
5) En solution.ipynb deben estar resueltas las respuestas a todas las preguntas del desafío
6) Dentro del repositorio deben estar todos los archivos necesarios para que los evaluadores puedan clonar y luego correr tu
notebook sin problemas
7) Una copia de tu CV (curriculum vitae) en formato .pdf en el repositorio



## Problema

El problema consiste en predecir la probabilidad de atraso de los vuelos que aterrizan o despegan del aeropuerto de Santiago de
Chile (SCL). Para eso les entregamos un dataset usando datos públicos y reales donde cada fila corresponde a un vuelo que
aterrizó o despegó de SCL. Para cada vuelo se cuenta con la siguiente información:

Fecha-I : Fecha y hora programada del vuelo.

Vlo-I : Número de vuelo programado.

Ori-I : Código de ciudad de origen programado.

Des-I : Código de ciudad de destino programado.

Emp-I : Código aerolínea de vuelo programado.

Fecha-O : Fecha y hora de operación del vuelo.

Vlo-O : Número de vuelo de operación del vuelo.

Ori-O : Código de ciudad de origen de operación

Des-O : Código de ciudad de destino de operación.

Emp-O : Código aerolínea de vuelo operado.

DIA : Día del mes de operación del vuelo.

MES : Número de mes de operación del vuelo.

AÑO : Año de operación del vuelo.

DIANOM : Día de la semana de operación del vuelo.

TIPOVUELO : Tipo de vuelo, I =Internacional, N =Nacional.

OPERA : Nombre de aerolínea que opera.

SIGLAORI : Nombre ciudad origen.

SIGLADES : Nombre ciudad destino.

### Desafío
1. ¿Cómo se distribuyen los datos? ¿Qué te llama la atención o cuál es tu conclusión sobre esto?
2. Genera las columnas adicionales y luego expórtelas en un archivo synthetic_features.csv :
○ temporada_alta : 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.
○ dif_min : diferencia en minutos entre Fecha-O y Fecha-I .
○ atraso_15 : 1 si dif_min > 15, 0 si no.
○ periodo_dia : mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche (entre 19:00 y 4:59), en base a
Fecha-I .
3. ¿Cómo se compone la tasa de atraso por destino, aerolínea, mes del año, día de la semana, temporada, tipo de vuelo?
¿Qué variables esperarías que más influyeran en predecir atrasos?
4. Entrena uno o varios modelos (usando el/los algoritmo(s) que prefieras) para estimar la probabilidad de atraso de un vuelo.
Siéntete libre de generar variables adicionales y/o complementar con variables externas.
5. Evalúa tu modelo. ¿Qué performance tiene? ¿Qué métricas usaste para evaluar esa performance y por qué? ¿Por qué
elegiste ese algoritmo en particular? ¿Qué variables son las que más influyen en la predicción? ¿Cómo podrías mejorar la
performance?
Aspectos a considerar
Orden y claridad al momento de plantear un análisis, idea, código, etc.
Creatividad para resolver el desafío.
Código versionado en Git.
No vamos a revisar excel, macros, códigos en R.
No vamos a revisar desafíos que no lleguen en la fecha indicada
Ante cualquier duda, deja explícitos tus supuestos
No vivimos en tu cabeza, trata de expresarte lo mejor posible para explicar tus decisiones y respuestas

## Paso 1: Lectura

In [ ]:
# importamos librerías importantes para la tarea 
import pandas as pd
import os

In [ ]:
# leemos la base cruda con pandas
raw = pd.read_csv('dataset_SCL.csv',
                  dtype={'Fecha-I':'str',
                         'Vlo-I':'str',
                         'Ori-I':'str',
                         'Des-I':'str',
                         'Emp-I':'str',
                         'Fecha-O':'str',
                         'Vlo-O':'str'})  # da algunos problemas porque Vlo-I parecía ser numérico pero no lo es. Ej: "405A"

In [ ]:
# raw = pd.read_csv('dataset_SCL.csv')

In [ ]:
# nuestra lectura de datos es un dataframe?
isinstance(raw, pd.DataFrame)

In [ ]:
# Sí lo es, pues lo leímos read_csv de pandas que la transforma en dataframe, luego:
df = raw

## Paso 2: Exploración

In [ ]:
# Tenemos 18 variables predictoras X1..X18 para nuestra variable respuesta Y (que, sospecho, debe ser la dif entre Fecha-I y Fecha-O)
df.head()

In [ ]:
df.info()

In [ ]:
# vamos a mirar unos primeros valores del df
df.head(n=100)

In [ ]:
# algunas estadísticas descriptivas
df.describe()
# no nos dice nada pq lo único numérico hasta ahora son DIA MES AÑO

## Paso 3: Pre-procesamiento de Valores faltantes y extremos

In [ ]:
# importamos el módulo ML necesario
from sklearn import preprocessing

In [ ]:
df.isna().sum()

Tenemos suerte, sólo la columna Vlo-O tiene un espacio en blanco que presumiblemente sería un número: 200.

Supuesto: Dado que Vlo-I es también 200. Podemos quitar la linea completa o bien extrapolarlo. Es razonable llenar el dato faltante.


In [ ]:
# Hay un dato faltante en la fila 6068 de la columna Vlo-O 
df.at[6068, 'Vlo-O']

In [ ]:
# lo llenamos
df.at[6068, 'Vlo-O'] = '200'

In [ ]:
# lo revisamos
df.at[6068, 'Vlo-O']

In [ ]:
# Luego ya no tenemos NANs
df.isna().sum()

In [ ]:
# df["date"] = df["Fecha-O"].astype("datetime64")
# df.groupby(df["date"].dt.month).count().plot(kind="bar")
# muy confuso, hagámoslo más sencillo.

In [ ]:
# Pequeño gráfico
df.groupby(df["OPERA"]).count().plot(kind="bar", )

In [ ]:
# veamos la distribución de los vuelos
# quiero contar los vuelos Vlo-O, a través 2017
# luego, agrupados por Operador.

freqByDate = df.groupby(['AÑO', 'MES']).size() 
print(freqByDate)

In [ ]:
# Agregados por Operador de vuelo y ordenados.
freqByOp = df.groupby(['OPERA']).size()
freqByOp.sort_values(axis=0)

Claramente Grupo Latam y Sky Airline tienen la mayor participación.

In [ ]:
# Veamos los valores extremos
# No hay mucho outlier pues gran parte de la data es categórica (tranformaremos esto más adelante con Label_Encoding)
# Y, si bien, la distribución de operadores está concentrada en pocos, no se consideran outliers.

### Pregunta 1: ¿Cómo se distribuyen los datos? ¿Qué te llama la atención o cuál es tu conclusión sobre esto?

En una 1era mirada tenemos: 

Una ventana temporal de un año (2017) de datos de vuelos de distintos operadores. Hay 2 datos de Enero 2018 que pueden confundirse en Enero de 2017 si hacemos agregación por MES solamente, pero para temas de distribución lo omitiremos.

1. Fecha-I y Fecha-O tienen formato "m/d/aaaa hh:mm"
2. I: Programado, lo planeado.
3. O: Operación, lo que realmente ocurrio.

Llama la atención que falta un dato en el registro posición 6068, que está en blanco. Pero bajo el supuesto de que, salvo 120 casos, Vlo-I = Vlo-O, lo llenamos con el número 200.

Mirando sólo descriptivamente la distribución de la cantidad de vuelos (operación) por fecha (agregación Año Mes) tenemos:

1. Se ve que la curva parte alta en Enero 2017 luego baja, supuestamente por temporada baja.
2. Se ve un peak en Julio, supuestamente vuelos internacionales que persiguen el calorcito de fin de año en el hemisferio Sur.
3. Hay otro peak en Septiembre, supuestamente por feriados alrededor del 18, porque parten a mitad de la 2da semana.
4. Se ve que desde Octubre 2017 vuelve a subir la curva hasta el máximo en Diciembre 2017, Navidad y verano en hemisferio Sur.

Me parece que habría que definir nuevas caracteristicas según lo solicitado, porque queremos modelar la diferencia entre las Fecha-I y Fecha-O, y en qué unidades medirlo. Definir también qué es un atraso. Además, según la distribución de vuelos en el año, podriamos decir algo sobre la temporada pues en verano (del hemisferio sur) hay una más vuelos que en invierno, hace sentido.

En el paso 4 vamos a definir lo que necesitemos.

## Paso 4: Definimos nuevas características

Definamos nuevas características (synthetic_features):

1. temporada_alta: 1 si Fecha-I está entre 15-Dic y 3-Mar, o 15-Jul y 31-Jul, o 11-Sep y 30-Sep, 0 si no.
2. dif_min: diferencia en minutos entre Fecha-O y Fecha-I .
3. atraso_15: 1 si dif_min > 15, 0 si no.
4. periodo_dia: mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche (entre 19:00 y 4:59), en base a Fecha-I.



### 4.1. Definimos temporada_alta

In [ ]:
# Se realizó 4.4.periodo_dia antes que 4.1.temporada_alta, pero son similares.
# Supuesto 1: Sin incluir extremos de las fechas.
# Supuesto 2: la condición (Fecha-I está entre 15-Dic y 3-Mar) supone que es antes de 3-Mar y después de 15-Dic,
# para que tenga sentido la condición. De otro modo, tendríamos que llamar temporada alta entre el 3-Mar a 15 Dic, lo cuál estaría erroneo.

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
#cambiamos las fechas a objeto datatime
from datetime import datetime

df['Fecha-O'] = pd.to_datetime(df['Fecha-O'], errors='coerce')
df['Fecha-I'] = pd.to_datetime(df['Fecha-I'], errors='coerce')

In [ ]:
# Fecha-I es type object, lo pasaremos a datetime
df['Fecha-I']

In [ ]:
# Lo pasamos a datetime para poder compararlo más abajo en las condiciones
#df['Fecha-I'] = pd.to_datetime(df['Fecha-I'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df['Fecha-I']

In [ ]:
import numpy as np
import datetime
# extraemos la parte dia mes de la fecha 
dateI = df['Fecha-I'].dt.date
#time = df['Fecha-I'].dt.strftime('%H:%M')
#dateI = pd.to_datetime(dateI, format='%Y-%m-%d %H:%M:%S')
dateI

In [ ]:
# vemos que funcione la comparación
dateI > datetime.date(2017, 1, 2)

In [ ]:
# usaremos np.select() igual que en 4.4.

In [ ]:
# creamos la lista de condiciones
conditions = [
    (dateI < datetime.date(2017, 3,3)),       # está antes de 3-Mar
    (dateI > datetime.date(2017, 12, 15)),    # está después de 15-Dic 
    (dateI > datetime.date(2017, 7, 15)) & (dateI < datetime.date(2017, 7, 31)), # está entre 15-Jul y 31-Jul
    (dateI > datetime.date(2017, 9, 11)) & (dateI < datetime.date(2017, 9, 30))  # está entre 11-Sep y 30-Sep
    ]

In [ ]:
# creamos la lista de valores para cada una de esas condiciones
values = ['1','1','1','1']

In [ ]:
# creamos la nueva columna y usamos np.select para asignar valores según nuestra lista como argumentos
df['temporada_alta'] = np.select(conditions, values)

In [ ]:
df['temporada_alta'] = df['temporada_alta']*1           # transformar falsos a 0
df['temporada_alta'] = df['temporada_alta'].astype(int) # lo quiero int32 igual que atraso_15

In [ ]:
# revisamos la columna para corroborar.
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df[['Fecha-I', 'temporada_alta']])

In [ ]:
# también revisar que haya quedado con tipo int32 
df.info()
# Pasemos al siguiente

### 4.2. Definimos dif_min

In [ ]:
# revisamos qué tipo de objeto son las fechas
df.info()

In [ ]:
#cambiamos las fechas a objeto datatime
from datetime import datetime

df['Fecha-O'] = pd.to_datetime(df['Fecha-O'], errors='coerce')
df['Fecha-I'] = pd.to_datetime(df['Fecha-I'], errors='coerce')

In [ ]:
# revisamos que hayan cambiado a objeto tipo datatime
df.info()

In [ ]:
# Luego de cambiar el tipo, hacemos diferencia de tiempo, medido en minutos
df['dif_min'] = (df['Fecha-O'] - df['Fecha-I']).dt.total_seconds() / 60

In [ ]:
# Finalmente, queda definida la diferencia:
df['dif_min']
# los números positivos indica la cantidad de minutos de atraso.
# notar que un número negativo implica que el vuelo llegó antes de lo esperado.

### 4.3. Definimos atraso_15

In [ ]:
# vamos a definir un atraso cuando dif_min sean más de 15 min:

df['atraso_15'] = df['dif_min'] > 15 # def condición
df['atraso_15'] = df['atraso_15']*1  # convierte los falses en 0 y trues en 1

In [ ]:
df['atraso_15']

### 4.4. Definimos periodo_dia

definimos: periodo_dia: mañana (entre 5:00 y 11:59), tarde (entre 12:00 y 18:59) y noche (entre 19:00 y 4:59), en base a Fecha-I.

quiero extraer hh:mm de cada fecha, y hacer condiciones.
El pseudo sería algo así:

si (Fecha-I > 05:00) AND (Fecha-I < 11:59) then periodo_dia == "mañana"

si (Fecha-I > 12:00) AND (Fecha-I < 18:59) then periodo_dia == "tarde"

si (Fecha-I > 19:00) OR (Fecha-I < 04:59) then periodo_dia == "noche"




In [ ]:
# if (df['Fecha-I'] > datetime.time(5, 00)) & (df['Fecha-I'] < datetime.time(11, 59)):
#    df['periodo_dia'] == "mañana"
    
# el problema acá es que datetime.time no es del mismo tipo que Fecha-I.
# pensemos... >>> 3 doritos después: podría ser con una lista de condiciones y usar np.select()

In [ ]:
import numpy as np
from datetime import datetime
# extraemos la parte tiempo de la fecha 
time = df['Fecha-I'].dt.time
#time = df['Fecha-I'].dt.strftime('%H:%M')
time

In [ ]:
import datetime
# comparamos según el pseudo anterior:
time > datetime.time(19, 00)
# estos sí se pueden comparar

In [ ]:
# creamos la lista de condiciones
conditions = [
    (time >= datetime.time(5, 00)) & (time <= datetime.time(11, 59)),
    (time >= datetime.time(12, 00)) & (time <= datetime.time(18, 59)),
    (time >= datetime.time(19, 00)),
    (time <= datetime.time(4, 59))
    ]

In [ ]:
# creamos la lista de valores para cada una de esas condiciones
values = ['mañana', 'tarde', 'noche', 'noche']

In [ ]:
# creamos la nueva columna y usamos np.select para asignar valores según nuestra lista como argumentos
df['periodo_dia'] = np.select(conditions, values)

In [ ]:
# revisamos la columna para corroborar. Hay 27 periodo_dia == 0, veamos por qué.
x = df.pivot_table(
    index=['periodo_dia'], columns="atraso_15", aggfunc="size", fill_value=0
).reset_index()
x.columns.name = None
print(x)

# Aps, fue porque estaba todo con símbolos estrictos (<, >), se solucionó cuando colocamos = pues la comparación es al minuto.
# Solucionado, ahora ya no aparecen los ceros.

## Paso 5: Exportamos las columnas creadas al archivo "synthetic_features.csv"

In [ ]:
df.head()

In [ ]:
# exportamos las columnas a un csv en la misma carpeta, ordenados con el mismo índice que el dataset
export_columns = ['dif_min','atraso_15','periodo_dia','temporada_alta']
df.loc[:,export_columns].to_csv('synthetic_features.csv')


In [ ]:
# si no se pueden nombrar
# export_columns = [19:22]
# df.iloc[:,export_columns].to_csv('new.csv')

## Paso 6: Preguntas
1. ¿Cómo se compone la tasa de atraso por destino, aerolínea, mes del año, día de la semana, temporada, tipo de vuelo?
2. ¿Qué variables esperarías que más influyeran en predecir atrasos?

In [ ]:
# miremos el dataframe con los nuevos features
df.head()
# En mi opinión sería útil mirar esto con alguna herramienta de BI y visualización como PowerBI, se puede llegar a insights rápido y muchas veces sirven para elegir cómo modelar.

In [ ]:
# Preguntémonos por: destino, aerolínea, mes del año, día de la semana, temporada, tipo de vuelo
# Preguntémonos por: SIGLADES, OPERA, MES, DIANOM, temporada_alta, TIPOVUELO
# Hagamos las agregaciones para entender qué sucede:

In [ ]:
By_SIGLADES = df.groupby(['atraso_15','SIGLADES']).size() 
# By_SIGLADES.sort_values(axis=0)
print(By_SIGLADES)

In [ ]:
By_OPERA = df.groupby(['atraso_15','OPERA']).size()
# By_OPERA.sort_values(axis=0)
print(By_OPERA)

In [ ]:
By_DIANOM = df.groupby(['atraso_15','DIANOM']).size() 
# By_DIANOM.sort_values(axis=0)

In [ ]:
By_MES = df.groupby(['atraso_15','MES']).size() # ordenado por mes
By_MES
# Hay una acumulación de atrasos en Jul, y Oct Nov y Dic

In [ ]:
By_temporada_alta = df.groupby(['atraso_15','temporada_alta']).size() 
# By_temporada_alta.sort_values(axis=0)
print(By_temporada_alta)
# la mayor cantidad de atrasos ocurre en temporada baja, más del doble.

In [ ]:
By_TIPOVUELO = df.groupby(['atraso_15','TIPOVUELO']).size() 
# By_TIPOVUELO.sort_values(axis=0)
print(By_TIPOVUELO)
# Los atrasos ocurren más en vuelos internacionales (7048, un 56% del total de atrasos)

In [ ]:
# otra forma para ver multivariable TIPOVUELO, temporada_alta
# Aquí vemos distintas agregaciones de atraso_15, agrupado por temporada_alta agrupado y TIPOVUELO.

x = df.pivot_table(
    index=['temporada_alta', 'TIPOVUELO'], columns="atraso_15", aggfunc="size", fill_value=0
).reset_index()
x.columns.name = None
print(x)

# Por ejemplo, vemos que la mayor cantidad de atrasos (4635, un 38% del total) ocurre en temporada baja en vuelos internacionales.
# También, que la menor cantidad de atrasos (1721, sólo 14% del total) ocurre en temporada alta en vuelos nacionales.

In [ ]:
# otra forma para ver los atrasos por temporada_alta y DIANOM.
x = df.pivot_table(
    index=['temporada_alta','DIANOM'], columns="atraso_15", aggfunc="size", fill_value=0
).reset_index()
x.columns.name = None
print(x)

# Vie, Lun y Jue hay más atrasos en temporada baja.
# Vie, Jue, Mar y Mie hay más atrasos en temporada alta. Aunque no hay mucha dif entre estos últimos.

In [ ]:
# otra forma para ver los atrasos por TIPOVUELO, OPERA.
x = df.pivot_table(
    index=['TIPOVUELO','OPERA'], columns="atraso_15", aggfunc="size", fill_value=0
).reset_index()
x.columns.name = None
print(x)

# tiene sentido porque también hay más operadores internacionales.

In [ ]:
# otra forma para ver los atrasos por TIPOVUELO, periodo_dia.
x = df.pivot_table(
    index=['TIPOVUELO','periodo_dia'], columns="atraso_15", aggfunc="size", fill_value=0
).reset_index()
x.columns.name = None
print(x)

# tiene sentido porque también hay más operadores internacionales.

In [ ]:
df.head()

In [ ]:
# probemos graficar df.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# La matriz de correlación sin Encoding luciría así:
df_corrmatrix = df.corr()
plt.figure(figsize=(20, 6))
sns.heatmap(df_corrmatrix, vmax=1, annot=True, linewidths=.5)
plt.xticks(rotation=30, horizontalalignment='right')
plt.show()

Mirando la matrix del dataframe df sin Encoding, tenemos:
1. Hay una relación entre atraso_15 y df_min, pero la diferencia en minutos fue usada para construir atraso_15 que es nuestra variable respuesta (Y), por tanto obviamente mostrará fuerza (artificial) en su relación, no es lo que buscamos.
2. MES tiene algo de correlación (0.083) directa, pero no es suficiente.
3. Hay una relación artificial entre temporada_alta y DIA, por motivos de construcción. Tal como en 1.
4. Hay una relación artificial entre temporada_alta y MES, por motivos de construcción. Tal como en 1.

¿Cómo se compone la tasa de atraso por destino, aerolínea, mes del año, día de la semana, temporada, tipo de vuelo?
¿Qué variables esperarías que más influyeran en predecir atrasos?

La variable más influyente parece ser: MES

DE las variables sintéticas, la más influyente parece ser: dif_min, pero esta la usamos para construir atraso_15, así que no sirve.

Pero también parece razonable pensar que la hora de salida del vuelo tenga que ver con los retrasos.

También parece razonable pensar que los vuelos anteriores retrazan a los sucesivos.

Cabe la pregunta:
¿Existe relación entre la hora de salida de un vuelo y su retrasos?


In [ ]:
# bueno para comparaciones.
# breakfast_time = now.replace( hour=7, minute=30, second=0, microsecond=0 )
# lunch_time = now.replace( hour=12, minute=30, second=0, microsecond=0 )
# coffee_break = now.replace( hour=16, minute=00, second=0, microsecond=0 )

# breakfast_time <= lunch_time <= coffee_break

In [ ]:
# Útil
# df['Fecha-I'] = pd.to_datetime(df['Fecha-I'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
# Útil
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd

# df
# df['Date'] = pd.date_range(start='01/01/2017', end='31/12/2017')
# df['Value'] = np.random.randint(low=5, high=100, size=len(df))
# df.set_index('Date', inplace=True)

# df.plot()
# plt.show()

# df.plot(kind='bar')
# plt.show()

## Paso 7: Label Encoding

Transformaremos las siguientes características a tipo numérico mediante Label_Encoding:

EMP-I, ORI-O, DES-O, EMP-O, DIANOM, TIPOVUELO, OPERA, SIGLAORI, SIGLADES, periodo_dia

Sabemos que EMP-O = OPERA.

In [ ]:
# Para ello habrá que hacer un Label_Encoding. Quiero transformar en número algunas variables: 

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# le = preprocessing.LabelEncoder()

In [ ]:
# df['label'] = le.fit_transform(df['label'])

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# debería droppear Fecha-I y Fecha-O para analizar sólo los numéricos.
# df.drop(labels=['Fecha-I', 'Fecha-O'], axis=1)

In [ ]:
# In order to convert data-frame column containing text to encoded values,
# just use the function text_to_numbers it returns a dictonary of LabelEncoding.
# Key is the column name that column LabelEncoder() as a value.
# https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn
# S.Rucinski

from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

def text_to_numbers(df):
        le_dict = dict()
        for i in df.columns:
            if df[i].dtype not in ["bool", "float32", "float64", "int32", "int64", "datetime64[ns]"]: # no convierte las features de estos type
                le_dict[i] = preprocessing.LabelEncoder()
                df[i] = le_dict[i].fit_transform(df[i])
    
        return df, le_dict

In [ ]:
# Y esta es f^(-1) de la f anterior. Es decir, es la función que nos permitirá recuperar el dataframe original.
def numbers_to_text(df, le_dict):
        for i in le_dict.keys():
            df[i] = le_dict[i].inverse_transform(df[i])
    
        return df

In [ ]:
# Usamos la función para el Encoding.
# Notar que en la función text_to_numbers() no convertimos explícitamente algunas características, 
dfle2 = text_to_numbers(df)
print(dfle2)

In [ ]:
type(dfle2)

In [ ]:
# Como df_le es una 2-tupla pq tiene el diccionario en su 2do elemento, extraemos el 1er elemento que es el df ya transformado,
# y lo dejamos como nuestro dataframe de insumo, lo llamamos dfle.

In [ ]:
le_df = dfle2[0]
le_dict = dfle2[1]
le_df.info()

In [ ]:
# Vemos como queda los tipos del dataframe. Todos int32 e int64, las fechas intactas
# y el flotante que es dif_min también intacto.
le_df.info()

In [ ]:
# Se ve mucho más como comida de algoritmos
le_df.head()

In [ ]:
print(le_df.nunique())

In [ ]:
print(le_dict)
type(le_dict)

In [ ]:
# Para devolvernos sólo tenemos que usar la función y darle 2 argumentos:
# El dataframe convertido (dfle) y el diccionario del LE (le_dict).
# Pero por ahora lo dejaremos comentado.

# numbers_to_text(le_df, le_dict)

## Paso 8: Gráficos

In [ ]:
# Veamos unos pocos gráficos
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.head()

In [ ]:
# Excluimos algunas columnas
#df2 = df.loc[:, ~df.columns.isin(['ORI-I', 'ORI-O', 'SIGLAORI'])]
#df2 = df.drop(['ORI-I', 'ORI-O', 'SIGLAORI'], axis=1)

In [ ]:
# La matriz de correlación CON Encoding luciría así:

df_corrmatrix = df.corr()
plt.figure(figsize=(20, 6))
sns.heatmap(df_corrmatrix, vmax=1, annot=True, linewidths=.5)
plt.xticks(rotation=30, horizontalalignment='right')
plt.show()

Ignoremos por el momento SIGLAORI, ORI-I, ORI-O que se nos pasó en el Encoder y lo dejó en 0.
También ignoremos, como vimos antes, dif_min porque lo usamos para construir atraso_15.
Miremos la correlación entre atraso_15 (la que nos importa) y TIPOVUELO, hay una correlación inversa (-0.096), pero tiene poca fuerza.
Miremos la correlación entre atraso_15 (la que nos importa) y MES, hay una correlación directa (0.083), pero tiene poca fuerza.

In [ ]:
# definimos un boxplot para mirar graficamente.
#bp1 = le_df[['atraso_15','periodo_dia']]
#sns.boxplot(x="variable", y="value", data=pd.melt(bp1)) # melt despivotea
#plt.show()


## Paso 9: Modelo

In [ ]:
# posibles modelos a priori dado que la Y (atraso_15) es binaria: Logistic, RandomForest, XGBoosting

In [ ]:
# XGBoosting